# Notebook 3: Rotate Select Mito Contigs<a class="tocSkip">

**We have to rotate mito contigs for two of the samples. If we don't do this then the next notebook doesn't have the correct data from minimap2 (since the hits are fragmented).**
    
*Note that I won't update the data table in the notebook. I'll do it manually in the Terra UI.*
    
    
**The steps that we will take are:**
1. Import Statements & Global Variable Definitions
2. Load Data Table
5. Rotate Select Circular Mito Contigs

# Import Statements & Global Variable Definitions

## Load Python packages
----

In [1]:
%%capture 
import terra_notebook_utils as tnu
import terra_pandas as tp
import os
import io
import gzip
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq, Alphabet

## Set Environment Variables

In [2]:
# Get the Google billing project name and workspace name
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_Reassembly
Workspace storage bucket: gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/


## Function Definitions

# Load Data Table

In [3]:
sample_df = tp.table_to_dataframe("clean_sample")

sample_df.head()

,mat_masked_fa,mito_against_ref_paf,pat_masked_cleaned_fa,mat_masked_cleaned_fa,pat_mito_contig_ls,pat_contam_results,mat_mito_contig_ls,hifiasm_mat_fa,all_mito_contigs,hifiasm_pat_fa,sample_name,pat_drop_contigs,mat_drop_contigs,pat_masked_fa,mat_contam_results
clean_sample_id,,,,,,,,,,,,,,,
HG002_downsampled,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/2...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/8...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/2...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...
HG002_full_v0.14,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/f...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/8...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/f...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...
HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/8...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...
HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/8...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...
HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/m...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/8...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/k...


# Rotate Select Circular Mito Contigs

If we don't do this for HG02559 and HG03098 then the minimap2 results for these samples are fragmented. Technically we should be looking for circular contigs in all samples, but the rest of the samples seem to have plenty of linear contigs to pull a full length MT contig from. These two samples only have circular contigs.

In [4]:
! mkdir rotate_mito
%cd rotate_mito

mkdir: cannot create directory ‘rotate_mito’: File exists
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/rotate_mito


## Rotate HG02559

In [5]:
sample_name = "HG02559"
sample_row  = sample_df.loc[sample_df.index == sample_name]

mt_contigs_fp = sample_row['all_mito_contigs'][0]
mt_contigs_fn = os.path.basename(mt_contigs_fp)

! gsutil cp {mt_contigs_fp} .

Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/85e90c15-bb68-4ea2-bfe2-56fd21fbb795/extractMitoContigs/0b95739b-748d-4c83-abf7-e905330dc40c/call-extractContigs/cacheCopy/HG02559.all_mito_contigs.fa...
/ [1 files][ 32.9 KiB/ 32.9 KiB]                                                
Operation completed over 1 objects/32.9 KiB.                                     


In [6]:
## Pulled this from the first attempt to cut the mito contigs
rotation      = 13282

**Pull fasta sequence and rotate, then add the rotated sequence to the original fa**

In [7]:
records = SeqIO.parse(mt_contigs_fn, 'fasta')

with open(mt_contigs_fn) as handle:
    for record in  SeqIO.parse(handle, "fasta"):
        if record.id == "h2tg000306c":
            ## Extract string, then rotate
            original_seq_str  = str(record.seq)
            rotated_seq_str   = original_seq_str[rotation:] + original_seq_str[:rotation]
            
            ## Write to SeqIO record
            record.seq  = Seq(rotated_seq_str, Alphabet.generic_dna)
            record.id   = "h2tg000306c-rotated"
            record.description = "h2tg000306c-rotated"
            
            ## There are two copies that are the same (different IDs): only need to write one
            break
            
with open("tmp.fa", "w") as output_handle:
    SeqIO.write(record, output_handle, "fasta")

In [8]:
! cat {mt_contigs_fn} tmp.fa > tmp && mv tmp {mt_contigs_fn}

In [9]:
! gsutil cp {mt_contigs_fn} {bucket}mito_work/manually_rotated/{mt_contigs_fn}

Copying file://HG02559.all_mito_contigs.fa [Content-Type=application/octet-stream]...
/ [1 files][ 49.4 KiB/ 49.4 KiB]                                                
Operation completed over 1 objects/49.4 KiB.                                     


## Rotate HG03098

In [10]:
sample_name = "HG03098"
sample_row  = sample_df.loc[sample_df.index == sample_name]

mt_contigs_fp = sample_row['all_mito_contigs'][0]
mt_contigs_fn = os.path.basename(mt_contigs_fp)

! gsutil cp {mt_contigs_fp} .

Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/85e90c15-bb68-4ea2-bfe2-56fd21fbb795/extractMitoContigs/02edf5da-32f6-423e-ba53-b38a5d4c345e/call-extractContigs/cacheCopy/HG03098.all_mito_contigs.fa...
/ [1 files][ 32.9 KiB/ 32.9 KiB]                                                
Operation completed over 1 objects/32.9 KiB.                                     


In [11]:
## Pulled this from the first attempt to cut the mito contigs
rotation      = 6252

In [12]:
records = SeqIO.parse(mt_contigs_fn, 'fasta')

with open(mt_contigs_fn) as handle:
    for record in  SeqIO.parse(handle, "fasta"):
        if record.id == "h2tg000243c":
            ## Extract string, then rotate
            original_seq_str  = str(record.seq)
            rotated_seq_str   = original_seq_str[rotation:] + original_seq_str[:rotation]
            
            ## Write to SeqIO record
            record.seq  = Seq(rotated_seq_str, Alphabet.generic_dna)
            record.id   = "h2tg000243c-rotated"
            record.description = "h2tg000243c-rotated"
            
            ## There are two copies that are the same (different IDs): only need to write one
            break
            
with open("tmp.fa", "w") as output_handle:
    SeqIO.write(record, output_handle, "fasta")

In [13]:
! cat {mt_contigs_fn} tmp.fa > tmp && mv tmp {mt_contigs_fn}

In [14]:
! gsutil cp {mt_contigs_fn} {bucket}mito_work/manually_rotated/{mt_contigs_fn}

Copying file://HG03098.all_mito_contigs.fa [Content-Type=application/octet-stream]...
/ [1 files][ 49.4 KiB/ 49.4 KiB]                                                
Operation completed over 1 objects/49.4 KiB.                                     
